In [1]:
import os
import sys

import semantic_kernel as sk

In [2]:

kernel = sk.Kernel()

In [3]:
import semantic_kernel.connectors.ai.open_ai as skaoai

In [4]:
deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
kernel.add_chat_service("azure_chat_competion_service", skaoai.AzureChatCompletion("GPT3TurboModel",endpoint,api_key=api_key,api_version = "2023-07-01-preview"))

In [5]:
from semantic_kernel.planning.basic_planner import BasicPlanner
planner = BasicPlanner()

In [6]:
base_skills_directory = '../../plugins'

In [7]:
custom_plugin = kernel.import_native_skill_from_directory(base_skills_directory , "APIPlugin")
writer_plugin = kernel.import_semantic_skill_from_directory(base_skills_directory, "WriterPlugin")
email_plugin = kernel.import_semantic_skill_from_directory(base_skills_directory, "EmailPlugin")
translate_plugin = kernel.import_semantic_skill_from_directory(base_skills_directory, "TranslatePlugin")

In [8]:
ask = """
Check the weather in Guangzhou and generate dressing tips based on the results and write them as travel weather reminder emails ，translate the results into spanish
"""
original_plan = await planner.create_plan_async(ask, kernel)

In [9]:
print(original_plan.generated_plan)

    {
        "input": "Guangzhou Weather",
        "subtasks": [
            {"function": "APIPlugin.WeatherFunction", "args": {"city": "Guangzhou"}},
            {"function": "WriterPlugin.Classification", "args": {"input": "dressing tips"}},
            {"function": "EmailPlugin.WeatherMail"},
            {"function": "TranslatePlugin.MultiLanguage", "args": {"language": "Spanish"}}
        ]
    }


In [10]:
await planner.execute_plan_async(original_plan, kernel)

'Tema: Consejos de vestimenta para viajar\n\nEstimados colegas,\n\nSegún el último pronóstico del tiempo, nos enfrentaremos a algunos cambios en el clima en los próximos días. Para asegurar nuestro viaje sea cómodo y seguro, me gustaría darles algunos consejos sobre qué ropa llevar.\n\nSe espera lluvia para mañana, así que recuerden llevar un paraguas y usar un abrigo impermeable. El jueves y viernes el clima se enfriará, así que preparen un abrigo o suéter abrigado. Se espera un clima despejado para el fin de semana, pero la temperatura podría bajar, así que recuerden llevar un abrigo ligero.\n\nAdemás, debido a los grandes cambios en el clima, por favor estén atentos a las condiciones de viaje y ajusten su vestimenta según sea necesario. Si hay alguna situación de emergencia, por favor contáctennos de inmediato, haremos todo lo posible para ayudar.\n\n¡Espero que tengan un viaje agradable, gracias!\n\nSaludos,\n\n[Tu nombre]'